In [6]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [7]:
import folium
import pandas as pd

In [8]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [10]:
import io

In [11]:
# Import necessary libraries
import js
import folium
import pandas as pd

# Download the spacex_launch_geo.csv file
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await js.fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

# Select relevant sub-columns: 'Launch Site', 'Lat(latitude)', 'Long(longitude)', 'class'
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

# NASA Johnson Space Center coordinate
nasa_coordinate = [29.559684888503615, -95.08309719339758]

# Create a map centered on NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

#Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson'))
# create a blue circle at NASA Johnson Space Center's coordinate with an icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

# Initial the map with a given center latitude and longitude
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    folium.map.Marker(coordinate, icon=folium.DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'], )).add_to(site_map)

# Display the map
site_map

In [12]:
# Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value

# Function to assign color based on class (0 or 1)
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

# Import MarkerCluster Library
from folium.plugins import MarkerCluster

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    coordinate = [record['Lat'], record['Long']]
    marker = folium.Marker(coordinate, icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Display the map
site_map

In [14]:
# Import necessary libraries
import io
import js
import folium
import pandas as pd
from folium.plugins import MousePosition
from math import sin, cos, sqrt, atan2, radians
from folium.features import DivIcon

# Download the spacex_launch_geo.csv file
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await js.fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

# Select relevant sub-columns: 'Launch Site', 'Lat(latitude)', 'Long(longitude)', 'class'
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

# Initial the map with a given center latitude and longitude
nasa_coordinate = [29.559684888503615, -95.08309719339758]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


# Iterate through each launch site and calculate the distance to the coastline
for index, launch_site in launch_sites_df.iterrows():
    launch_site_lat = launch_site['Lat']
    launch_site_lon = launch_site['Long']

    # Find coordinate of the closest coastline (You need to manually find this using MousePosition)
    # e.g.,: Lat: 28.56367  Lon: -80.57163
    coastline_lat = 28.56367   # Replace with the actual coastline latitude you find
    coastline_lon = -80.57163  # Replace with the actual coastline longitude you find

    # Calculate the distance between the launch site and the coastline
    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

    # Create coordinates for the launch site
    coordinate = [launch_site_lat, launch_site_lon]

    # Create and add a folium.Marker on your selected closest coastline point on the map
    distance_marker = folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
    site_map.add_child(distance_marker)

    # Draw a PolyLine between a launch site to the selected coastline point
    # Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
    lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
    site_map.add_child(lines)

# Display the map
site_map